<a href="https://colab.research.google.com/github/Gaurav-phatkare/arXiv-Paper-Abstracts-Mutli-Label-Classification-Using-Deep-Learning/blob/main/Multilabel_Text_Classification_of_Research_Papers_different_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Aug  5 16:03:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [3]:
!pip install kaggle

In [4]:
!mkdir ~/.kaggle

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d spsayakpaul/arxiv-paper-abstracts

 61% 27.0M/44.6M [00:00<00:00, 157MB/s] 
100% 44.6M/44.6M [00:00<00:00, 175MB/s]


In [8]:
!unzip /content/arxiv-paper-abstracts.zip

Archive:  /content/arxiv-paper-abstracts.zip
  inflating: arxiv_data.csv          
  inflating: arxiv_data_210930-054931.csv  


In [9]:
df = pd.read_csv("/content/arxiv_data.csv")

In [10]:
df.shape

(51774, 3)

In [11]:
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [12]:
df.iloc[4].summaries,df.iloc[4].terms

('To ensure safety in automated driving, the correct perception of the\nsituation inside the car is as important as its environment. Thus, seat\noccupancy detection and classification of detected instances play an important\nrole in interior sensing. By the knowledge of the seat occupancy status, it is\npossible to, e.g., automate the airbag deployment control. Furthermore, the\npresence of a driver, which is necessary for partially automated driving cars\nat the automation levels two to four can be verified. In this work, we compare\ndifferent statistical methods from the field of image segmentation to approach\nthe problem of background-foreground segmentation in camera based interior\nsensing. In the recent years, several methods based on different techniques\nhave been developed and applied to images or videos from different\napplications. The peculiarity of the given scenarios of interior sensing is,\nthat the foreground instances and the background both contain static as well as\

In [13]:
df['titles'].duplicated().sum()

12802

In [14]:
df.drop_duplicates(inplace = True)

In [15]:
df.shape

(38991, 3)

In [16]:
import re
def Text_cleaning(text):
  text = re.sub(r"\n" , " ", text)

  text = re.sub(r"\s+" , " ", text)

  text = text.lower()
  text = text.replace('\n'," ")

  return text



In [17]:
Text_cleaning(df.iloc[4].summaries)

'to ensure safety in automated driving, the correct perception of the situation inside the car is as important as its environment. thus, seat occupancy detection and classification of detected instances play an important role in interior sensing. by the knowledge of the seat occupancy status, it is possible to, e.g., automate the airbag deployment control. furthermore, the presence of a driver, which is necessary for partially automated driving cars at the automation levels two to four can be verified. in this work, we compare different statistical methods from the field of image segmentation to approach the problem of background-foreground segmentation in camera based interior sensing. in the recent years, several methods based on different techniques have been developed and applied to images or videos from different applications. the peculiarity of the given scenarios of interior sensing is, that the foreground instances and the background both contain static as well as dynamic eleme

In [18]:
df['cleaned_summary'] = df['summaries'].apply(Text_cleaning)

In [19]:
df.head()

,titles,summaries,terms,cleaned_summary
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']",stereo matching is one of the widely used tech...
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']",the recent advancements in artificial intellig...
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']","in this paper, we proposed a novel mutual cons..."
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV'],consistency training has proven to be an advan...
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']","to ensure safety in automated driving, the cor..."


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38991 entries, 0 to 51772
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titles           38991 non-null  object
 1   summaries        38991 non-null  object
 2   terms            38991 non-null  object
 3   cleaned_summary  38991 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [21]:
df.terms.value_counts()

['cs.CV']                                          12753
['cs.LG', 'stat.ML']                                4076
['cs.LG']                                           2047
['cs.CV', 'cs.LG']                                  1488
['cs.LG', 'cs.AI']                                  1207
                                                   ...  
['cs.LG', 'cs.CL', 'cs.HC', 'stat.ML']                 1
['cs.LG', 'cs.AI', 'cs.CL', 'cs.PL', 'stat.ML']        1
['cs.LG', 'cs.CL', 'stat.ME', 'stat.ML']               1
['cs.LG', 'cs.CL', 'cs.LO', 'stat.ML']                 1
['stat.ML', 'cs.CV', 'cs.LG', 'q-bio.QM']              1
Name: terms, Length: 3157, dtype: int64

In [22]:
new_df = df.groupby('terms').filter(lambda x: len(x) > 1)

In [23]:
new_df.terms.value_counts()

['cs.CV']                                                         12753
['cs.LG', 'stat.ML']                                               4076
['cs.LG']                                                          2047
['cs.CV', 'cs.LG']                                                 1488
['cs.LG', 'cs.AI']                                                 1207
                                                                  ...  
['cs.CV', 'cs.LG', 'eess.IV', 'eess.SP', 'stat.ML']                   2
['cs.LG', 'cs.CL', 'cs.CV', 'cs.NE', 'stat.ML']                       2
['stat.ML', 'cs.IT', 'cs.LG', 'math.IT', 'math.ST', 'stat.TH']        2
['cs.CV', 'cs.CL', 'cs.LG', 'eess.AS', 'eess.IV']                     2
['cs.LG', 'G.3']                                                      2
Name: terms, Length: 836, dtype: int64

In [24]:
len(new_df.iloc[2].terms),new_df.iloc[2].terms

(18, "['cs.CV', 'cs.AI']")

In [25]:
new_df['terms'].values[:5]

array(["['cs.CV', 'cs.LG']", "['cs.CV', 'cs.AI', 'cs.LG']",
       "['cs.CV', 'cs.AI']", "['cs.CV']", "['cs.CV', 'cs.LG']"],
      dtype=object)

## convert the string labels to lists of strings

In [26]:
import ast
new_df['terms'] = new_df['terms'].apply(lambda x: ast.literal_eval(x))

In [27]:
len(new_df.iloc[2].terms),new_df.iloc[2].terms

(2, ['cs.CV', 'cs.AI'])

In [28]:
new_df['terms'].values[:5]

array([list(['cs.CV', 'cs.LG']), list(['cs.CV', 'cs.AI', 'cs.LG']),
       list(['cs.CV', 'cs.AI']), list(['cs.CV']),
       list(['cs.CV', 'cs.LG'])], dtype=object)

## Spliting into train test split

In [29]:
from sklearn.model_selection import train_test_split

def split_dataset(data, train_ratio=0.8, test_ratio=0.1, val_ratio=0.1, random_seed=42):

    train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=random_seed)
    test_data, val_data = train_test_split(temp_data, train_size=test_ratio / (test_ratio + val_ratio),
                                           random_state=random_seed)

    return train_data, test_data, val_data


train_data, test_data, val_data = split_dataset(new_df, train_ratio=0.8, test_ratio=0.1, val_ratio=0.1)


In [30]:
len(train_data),len(test_data),len(val_data)

(29336, 3667, 3667)

### Multi-label Binarization

In [31]:
tf.keras.layers.StringLookup(output_mode = 'multi_hot')

In [32]:
terms = tf.ragged.constant(train_data['terms'].values)
lookup = tf.keras.layers.StringLookup(output_mode = 'multi_hot')
lookup.adapt(terms)
vocab = lookup.get_vocabulary()

In [33]:
terms[:5]

<tf.RaggedTensor [[b'cs.CV', b'cs.LG', b'eess.IV'], [b'cs.CV', b'cs.AI', b'stat.ML'],
 [b'cs.LG'], [b'cs.CV'], [b'cs.CV', b'cs.AI']]>

In [34]:
vocab

['[UNK]',
 'cs.CV',
 'cs.LG',
 'stat.ML',
 'cs.AI',
 'eess.IV',
 'cs.RO',
 'cs.CL',
 'cs.NE',
 'cs.CR',
 'math.OC',
 'eess.SP',
 'cs.GR',
 'cs.SI',
 'cs.MM',
 'cs.SY',
 'cs.IR',
 'eess.SY',
 'cs.MA',
 'cs.HC',
 'cs.DC',
 'math.IT',
 'cs.IT',
 'cs.CY',
 'stat.AP',
 'stat.TH',
 'math.ST',
 'stat.ME',
 'eess.AS',
 'cs.SD',
 'q-bio.QM',
 'cs.DS',
 'q-bio.NC',
 'cs.NI',
 'cs.SE',
 'I.2.6',
 'stat.CO',
 'cs.CG',
 'cs.GT',
 'math.NA',
 'physics.chem-ph',
 'cs.NA',
 'cs.DB',
 'cs.LO',
 'cs.PL',
 '68T45',
 'q-bio.BM',
 'physics.comp-ph',
 'math.PR',
 'cs.AR',
 'cond-mat.dis-nn',
 'math.DS',
 'I.2.10',
 'cs.CE',
 'physics.data-an',
 'quant-ph',
 '68T05',
 'q-fin.ST',
 'cond-mat.stat-mech',
 'cs.DM',
 'physics.soc-ph',
 'cs.CC',
 'astro-ph.IM',
 'I.4.6',
 'q-bio.GN',
 'econ.EM',
 'physics.med-ph',
 'cs.PF',
 'I.4.8',
 'q-fin.TR',
 'physics.ao-ph',
 '68U10',
 'math.AT',
 'I.2',
 'hep-ex',
 'cs.FL',
 'cond-mat.mtrl-sci',
 'I.5.4',
 'I.4',
 'physics.optics',
 'physics.geo-ph',
 'math.AP',
 'I.4; I.5

In [35]:
def invert_multi_hot(encoded_labels):
  hot_indices = np.argwhere(encoded_labels == 1.0)[:,0]
  return np.take(vocab, hot_indices)

In [36]:
import numpy as np
ind = np.argwhere([1,0,0,1,0])[:,0]
np.take(['a','b','c','d','e'],ind)

array(['a', 'd'], dtype='<U1')

**Try multi hot with some sample**

In [37]:
sample_label = train_data['terms'].iloc[0]
sample_label

['cs.CV', 'cs.LG', 'eess.IV']

In [38]:
label_binarized = lookup(sample_label)
label_binarized

<tf.Tensor: shape=(151,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)>

**Invert binarized labels to sample labels**

In [39]:
invert_multi_hot(label_binarized)

array(['cs.CV', 'cs.LG', 'eess.IV'], dtype='<U40')

In [40]:
def length(x):
  x = x.split(" ")
  return len(x)

In [41]:
train_data['cleaned_summary'].apply(length).describe()

count    29336.000000
mean       171.597491
std         45.364715
min          5.000000
25%        140.000000
50%        169.000000
75%        201.000000
max        498.000000
Name: cleaned_summary, dtype: float64

##Preprocessing the data

In [42]:
max_length = 250
batch_size = 32
padding_token = '<pad>'
auto = tf.data.AUTOTUNE

In [43]:
tf.ragged.constant(train_data['terms'].values)

<tf.RaggedTensor [[b'cs.CV', b'cs.LG', b'eess.IV'], [b'cs.CV', b'cs.AI', b'stat.ML'],
 [b'cs.LG'], ..., [b'cs.CV', b'cs.CL'], [b'cs.CV'], [b'cs.CV', b'cs.CL']]>

In [44]:
lookup([b'cs.CV', b'cs.AI', b'stat.ML']).numpy()

array([0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [45]:
vocabulary = set()
train_data['cleaned_summary'].str.split().apply(vocabulary.update)
vocabulary_size = len(vocabulary)
print(vocabulary_size)

141608


In [46]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens = vocabulary_size,
                                    output_sequence_length = max_length)

In [47]:
# train_data['cleaned_summary'].tolist()

In [48]:
text_vectorizer.adapt(train_data['cleaned_summary'].tolist())

In [49]:
import random

target_sentance = random.choice(train_data['cleaned_summary'].tolist())
print(f'text:\n{target_sentance}')
print(f'text_length:\n{length(target_sentance)}')
print(f'\nVectorized text: {text_vectorizer([target_sentance])}')

text:
a semi-parametric, non-linear regression model in the presence of latent variables is introduced. these latent variables can correspond to unmodeled phenomena or unmeasured agents in a complex networked system. this new formulation allows joint estimation of certain non-linearities in the system, the direct interactions between measured variables, and the effects of unmodeled elements on the observed system. the particular form of the model adopted is justified, and learning is posed as a regularized empirical risk minimization. this leads to classes of structured convex optimization problems with a "sparse plus low-rank" flavor. relations between the proposed model and several common model paradigms, such as those of robust principal component analysis (pca) and vector autoregression (var), are established. particularly in the var setting, the low-rank contributions can come from broad trends exhibited in the time series. details of the algorithm for learning the model are prese

In [50]:

# get the config of our text vectorizer
text_vectorizer.get_config()


{'name': 'text_vectorization',
 'trainable': True,
 'dtype': 'string',
 'batch_input_shape': (None,),
 'max_tokens': 141608,
 'standardize': 'lower_and_strip_punctuation',
 'split': 'whitespace',
 'ngrams': None,
 'output_mode': 'int',
 'output_sequence_length': 250,
 'pad_to_max_tokens': False,
 'sparse': False,
 'ragged': False,
 'vocabulary': None,
 'idf_weights': None,
 'encoding': 'utf-8',
 'vocabulary_size': 81092}

In [51]:
# custom text embedding

text_vocab = text_vectorizer.get_vocabulary()
print(f"Number of words in vocabulary: {len(text_vocab)}"),
print(f"Most common words in the vocabulary: {text_vocab[:5]}")
print(f"Least common words in the vocabulary: {text_vocab[-5:]}")

Number of words in vocabulary: 81092
Most common words in the vocabulary: ['', '[UNK]', 'the', 'of', 'and']
Least common words in the vocabulary: ['000168', '00013acer', '00010', '00003', '00002']


In [52]:
import tensorflow as tf

token_embed = tf.keras.layers.Embedding(input_dim = len(text_vocab),
                                        output_dim = 256,
                                        mask_zero = True,
                                        name = 'token_embedding')

In [53]:
print(f'Target sentance:\n{target_sentance}\n\n')
print(f'Vectorized target sentance:\n{text_vectorizer([target_sentance])}\n\n')
print(f'shape of vectorized sentance:{text_vectorizer([target_sentance]).shape}\n\n')
print(f'embedded target sentance:\n{token_embed(text_vectorizer([target_sentance]))}\n\n')
print(f'shape of embedded sentance:{token_embed(text_vectorizer([target_sentance])).shape}')

Target sentance:
a semi-parametric, non-linear regression model in the presence of latent variables is introduced. these latent variables can correspond to unmodeled phenomena or unmeasured agents in a complex networked system. this new formulation allows joint estimation of certain non-linearities in the system, the direct interactions between measured variables, and the effects of unmodeled elements on the observed system. the particular form of the model adopted is justified, and learning is posed as a regularized empirical risk minimization. this leads to classes of structured convex optimization problems with a "sparse plus low-rank" flavor. relations between the proposed model and several common model paradigms, such as those of robust principal component analysis (pca) and vector autoregression (var), are established. particularly in the var setting, the low-rank contributions can come from broad trends exhibited in the time series. details of the algorithm for learning the mode

In [54]:
# def make_dataset(dataframe, is_train=True):
#   labels = tf.ragged.constant(dataframe['terms'].values)
#   label_binarized = lookup(labels).numpy()
#   dataset = tf.data.Dataset.from_tensor_slices(
#       (dataframe['cleaned_summary'].values, label_binarized)
#   )
#   dataset = dataset.shuffle(batch_size * 10) if is_train else dataset
#   return dataset.batch(batch_size)

In [55]:
train_sentances = train_data['cleaned_summary'].tolist()
val_sentances = val_data['cleaned_summary'].tolist()
test_sentances = test_data['cleaned_summary'].tolist()

In [56]:
def make_dataset(dataframe, is_train=True):
  labels = tf.ragged.constant(dataframe['terms'].values)
  label_binarized = lookup(labels).numpy()
  dataset = tf.data.Dataset.from_tensor_slices(
      (dataframe['cleaned_summary'].tolist(), label_binarized)
  )
  dataset = dataset.shuffle(batch_size * 10) if is_train else dataset
  return dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [57]:
train_dataset = make_dataset(train_data, is_train= True)
val_dataset = make_dataset(val_data, is_train= False)
test_dataset = make_dataset(test_data, is_train= False)

In [58]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 151), dtype=tf.float32, name=None))>

In [59]:
#model 1 : Conv1D witth token embeddings

import tensorflow as tf
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype = tf.string)
text_vectors = text_vectorizer(inputs)

token_embeddings = token_embed(text_vectors)

x = layers.Conv1D(64, kernel_size = 5, padding = 'same',activation = 'relu')(token_embeddings)
x = layers.GlobalMaxPooling1D()(x)
outputs = layers.Dense(151, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs)

model_1.compile(loss="binary_crossentropy", # if your labels are integer form (not one hot) use sparse_categorical_crossentropy
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["binary_accuracy"])

In [60]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 250)              0         
 torization)                                                     
                                                                 
 token_embedding (Embedding)  (None, 250, 256)         20759552  
                                                                 
 conv1d (Conv1D)             (None, 250, 64)           81984     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 151)               9815  

In [61]:
# fit the model

model_1_history = model_1.fit(train_dataset,
                              steps_per_epoch = int(0.1*len(train_dataset)),
                              epochs = 5,
                              validation_data = val_dataset,
                              validation_steps = int(0.1*len(val_dataset)))

Epoch 1/5
91/91 [==============================] - 29s 207ms/step - loss: 0.2043 - binary_accuracy: 0.9557 - val_loss: 0.0279 - val_binary_accuracy: 0.9896
Epoch 2/5
91/91 [==============================] - 13s 147ms/step - loss: 0.0285 - binary_accuracy: 0.9891 - val_loss: 0.0268 - val_binary_accuracy: 0.9899
Epoch 3/5
91/91 [==============================] - 11s 127ms/step - loss: 0.0279 - binary_accuracy: 0.9893 - val_loss: 0.0257 - val_binary_accuracy: 0.9910
Epoch 4/5
91/91 [==============================] - 9s 102ms/step - loss: 0.0262 - binary_accuracy: 0.9913 - val_loss: 0.0223 - val_binary_accuracy: 0.9934
Epoch 5/5
91/91 [==============================] - 4s 48ms/step - loss: 0.0238 - binary_accuracy: 0.9928 - val_loss: 0.0210 - val_binary_accuracy: 0.9938


In [62]:
model_1.evaluate(val_dataset)

115/115 [==============================] - 1s 5ms/step - loss: 0.0229 - binary_accuracy: 0.9932


[0.022912124171853065, 0.9931658506393433]

In [63]:
model_1.evaluate(test_dataset)

115/115 [==============================] - 1s 5ms/step - loss: 0.0231 - binary_accuracy: 0.9929


[0.023121260106563568, 0.9929313659667969]

In [64]:
model_1_pred_probs = model_1.predict(test_dataset)
model_1_pred_probs

115/115 [==============================] - 1s 4ms/step


array([[2.3348288e-04, 1.5300047e-01, 8.8243443e-01, ..., 1.3306540e-04,
        1.5361101e-04, 1.2416544e-04],
       [6.8642001e-04, 9.5211411e-01, 1.9600789e-01, ..., 1.1204163e-03,
        3.5037563e-04, 2.3970172e-04],
       [1.9567694e-04, 1.8580246e-01, 8.6793315e-01, ..., 1.3967442e-04,
        1.0635359e-04, 1.0752475e-04],
       ...,
       [2.8815512e-05, 9.8013157e-01, 1.4614795e-01, ..., 1.1768757e-04,
        4.8349470e-05, 3.7988782e-05],
       [7.3465271e-05, 9.8873866e-01, 1.3883586e-01, ..., 1.9512414e-04,
        5.8149024e-05, 3.5653869e-05],
       [1.8943110e-04, 2.1103632e-01, 8.3275074e-01, ..., 2.2949005e-04,
        1.8184482e-04, 1.9700613e-04]], dtype=float32)

In [65]:
model_1_pred_probs.shape

(3667, 151)

In [66]:
test_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None, 151), dtype=tf.float32, name=None))>

In [74]:
text, label = next(iter(test_dataset))

print(text[15])
print(label[15])
print(invert_multi_hot(label[15]))

tf.Tensor(b"in this work, we perform unsupervised learning of representations by maximizing mutual information between an input and the output of a deep neural network encoder. importantly, we show that structure matters: incorporating knowledge about locality of the input to the objective can greatly influence a representation's suitability for downstream tasks. we further control characteristics of the representation by matching to a prior distribution adversarially. our method, which we call deep infomax (dim), outperforms a number of popular unsupervised learning methods and competes with fully-supervised learning on several classification tasks. dim opens new avenues for unsupervised learning of representations and is an important step towards flexible formulations of representation-learning objectives for specific end-goals.", shape=(), dtype=string)
tf.Tensor(
[0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [75]:
test_pred_prob = model_1_pred_probs[15]

In [94]:
test_pred_prob.shape

(151,)

In [76]:
sorted_prob = sorted(zip(test_pred_prob, vocab), key = lambda pair : pair[0],reverse = True)

In [77]:
sorted_prob

[(0.88392806, 'cs.LG'),
 (0.6529016, 'stat.ML'),
 (0.271892, 'cs.AI'),
 (0.12052495, 'cs.CV'),
 (0.028467376, 'cs.NE'),
 (0.027929772, 'eess.IV'),
 (0.022925425, 'cs.CL'),
 (0.021290774, 'cs.RO'),
 (0.01558849, 'cs.CR'),
 (0.0127241025, 'eess.SP'),
 (0.011588188, 'math.OC'),
 (0.009846667, 'cs.SY'),
 (0.009747218, 'eess.SY'),
 (0.009248899, 'cs.MA'),
 (0.009191366, 'cs.DC'),
 (0.007531196, 'cs.IT'),
 (0.006561719, 'math.IT'),
 (0.0063598906, 'cs.IR'),
 (0.0061292406, 'cs.HC'),
 (0.005197547, 'stat.TH'),
 (0.005153965, 'stat.AP'),
 (0.0048100217, 'cs.SI'),
 (0.004398348, 'cs.CY'),
 (0.0040947264, 'cs.MM'),
 (0.0039578504, 'cs.DS'),
 (0.0037986974, 'cs.GR'),
 (0.0034507874, 'math.ST'),
 (0.0022313728, 'cs.CG'),
 (0.0019210523, 'math.NA'),
 (0.0016318634, 'eess.AS'),
 (0.0015051436, 'cs.SD'),
 (0.0014828136, 'q-bio.QM'),
 (0.0014694864, 'q-bio.NC'),
 (0.0013718328, 'stat.ME'),
 (0.0013613655, 'I.2.6'),
 (0.0013347517, 'cs.PL'),
 (0.0012743851, 'q-bio.BM'),
 (0.0012234383, 'cs.NA'),
 (0.00

In [122]:
i, j = next(iter(test_dataset))
print(i[10])
print(j[10])
print(invert_multi_hot(j[10]))

tf.Tensor(b'in this paper, we introduce an alternative approach, namely gen (genetic evolution network) model, to the deep learning models. instead of building one single deep model, gen adopts a genetic-evolutionary learning strategy to build a group of unit models generations by generations. significantly different from the wellknown representation learning models with extremely deep structures, the unit models covered in gen are of a much shallower architecture. in the training process, from each generation, a subset of unit models will be selected based on their performance to evolve and generate the child models in the next generation. gen has significant advantages compared with existing deep representation learning models in terms of both learning effectiveness, efficiency and interpretability of the learning process and learned results. extensive experiments have been done on diverse benchmark datasets, and the experimental results have demonstrated the outstanding performance 

In [124]:
test2 = model_1.predict(['in this paper, we introduce an alternative approach, namely gen (genetic evolution network) model, to the deep learning models. instead of building one single deep model, gen adopts a genetic-evolutionary learning strategy to build a group of unit models generations by generations. significantly different from the wellknown representation learning models with extremely deep structures, the unit models covered in gen are of a much shallower architecture. in the training process, from each generation, a subset of unit models will be selected based on their performance to evolve and generate the child models in the next generation. gen has significant advantages compared with existing deep representation learning models in terms of both learning effectiveness, efficiency and interpretability of the learning process and learned results. extensive experiments have been done on diverse benchmark datasets, and the experimental results have demonstrated the outstanding performance of gen compared with the state-of-the-art baseline methods in both effectiveness of efficiency.'])

1/1 [==============================] - 0s 52ms/step


In [125]:
tf.squeeze(test2).numpy()

array([1.4833226e-04, 2.1897906e-01, 8.8067156e-01, 5.6869400e-01,
       2.5819525e-01, 2.8648123e-02, 2.3463275e-02, 2.9541301e-02,
       3.0532857e-02, 1.6856827e-02, 1.2003086e-02, 2.2485455e-02,
       6.5567438e-03, 6.5173539e-03, 4.6726302e-03, 1.3176239e-02,
       7.0915180e-03, 1.6088450e-02, 1.2402689e-02, 5.9043872e-03,
       8.8249203e-03, 6.5711965e-03, 5.8132699e-03, 3.6789195e-03,
       4.5714420e-03, 5.5579236e-03, 4.9917102e-03, 1.3246312e-03,
       1.4781167e-03, 1.3845598e-03, 1.8278838e-03, 5.6364643e-03,
       2.0588855e-03, 7.3681248e-04, 1.2632380e-03, 1.1558816e-03,
       1.2893811e-03, 1.5540672e-03, 5.6393974e-04, 1.7857635e-03,
       1.4949925e-03, 1.2562447e-03, 2.6288794e-04, 2.4724888e-04,
       1.1515542e-03, 5.2776636e-04, 1.0039596e-03, 7.1733689e-04,
       7.3645799e-04, 7.1354269e-04, 4.5005360e-04, 4.2053167e-04,
       8.6873391e-04, 5.5483758e-04, 7.9683319e-04, 4.9321842e-04,
       5.0040265e-04, 4.7922184e-04, 3.7383410e-04, 2.5076899e

In [126]:
sorted_prob_01 = sorted(zip(tf.squeeze(test2).numpy(), vocab), key = lambda pair : pair[0],reverse = True)

In [127]:
sorted_prob_01[:10]

[(0.88067156, 'cs.LG'),
 (0.568694, 'stat.ML'),
 (0.25819525, 'cs.AI'),
 (0.21897906, 'cs.CV'),
 (0.030532857, 'cs.NE'),
 (0.0295413, 'cs.CL'),
 (0.028648123, 'eess.IV'),
 (0.023463275, 'cs.RO'),
 (0.022485455, 'eess.SP'),
 (0.016856827, 'cs.CR')]